In [1]:
import sys
sys.path.append("..")
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from project.object import Object
from project.tests import Tests
from project.botma import TMA
import time
from pathlib import Path
from datetime import datetime
import ast

In [2]:
tests = Tests('test3')
print("Время последнего моделирования:", datetime.now().strftime("%d/%m/%Y %H:%M"))

# Создаем наблюдателя
observer_x, observer_y, observer_course, observer_velocity = 0, 0, 0, 5
observer = Object('Наблюдатель', observer_x, observer_y, observer_course,
                observer_velocity, verbose=True)

# Моделирование траектории наблюдателя
observer.forward_movement(3 * 60)
observer.change_course(270, 'left', omega=0.5)
observer.forward_movement(5 * 60)
observer.change_course(90, 'right', omega=0.5)
observer.forward_movement(3 * 60)

Время последнего моделирования: 04/04/2021 07:47
Наблюдатель движется прямо по курсу 0.0° 180с
Наблюдатель перешёл на курс 270.0° за 180с
Наблюдатель движется прямо по курсу 270.0° 300с
Наблюдатель перешёл на курс 90.0° за 360с
Наблюдатель движется прямо по курсу 90.0° 180с


In [3]:
end_t = 420 # задаем Твыч

In [4]:
p0 = [0., 25., 90., 7.]
d_arr = [10., 20., 30., 40.]
std_arr = [0., 0.1, 0.2, 0.3, 0.5, 1.]
alg_arr = ['ММП v2', 'Метод N пеленгов']
def target_func(seed=None):
            rng = np.random.RandomState(seed)
            b = 0
            d = rng.uniform(5, 50)
            c = rng.uniform(0, 180)
            v = rng.uniform(3, 15)
            return [b, d, c, v]
result_df = pd.DataFrame(columns=['Algorithm', 'std', 'sr_0', 'sr_1', 'sr_2', 'sr_3', 'Ка', 'Nf', 'Nf_max', 'Iter', 'Iter_max', 't', 't_max'])
mean_columns = ['Ка', 'Nf', 'Iter', 't']
max_columns = ['Nf', 'Iter', 't']
Path("../tests/test3/{}/".format(end_t)).mkdir(parents=True, exist_ok=True)

In [5]:
%%time
tma = TMA(observer, end_t=end_t)
for std in std_arr:
    tma.noise_std = np.radians(std)
    for alg in alg_arr:
        dict_results = tma.swarm(algorithm_name=alg, n=1000, target_func=target_func, p0=p0, seeded=True)
        df = tests.get_df(dict_results)
        tests.save_df(df, name='../tests/test3/{}/'.format(end_t) + str(alg) + ' ' + str(std) + '.xlsx')
        res = pd.DataFrame({'Algorithm': [alg], 'std': [std]})
        for i in range(4):
            res['sr_' + str(i)] = [df['Успех'].apply(lambda x: x[i]).mean(axis=0)]
        res[mean_columns] = df[mean_columns].mean(axis=0)
        res[['Nf_max', 'Iter_max', 't_max']] = df[max_columns].apply(max, axis=0)
        result_df = result_df.append(res)

Wall time: 2min 23s


In [6]:
result_df.reset_index(drop=True)
result_df['Ка'] = result_df['Ка'].apply(np.sqrt)
tests.save_df(result_df, name='../tests/test3/{}/results.xlsx'.format(end_t))

In [7]:
result_df = pd.read_excel('../tests/test3/{}/results.xlsx'.format(end_t))
display_df = result_df.copy()
precisions = {'Ка' : '{0:.2f}', 'Nf' : '{0:.0f}', 'Nf_max': '{0:.0f}',\
'Iter' : '{0:.0f}', 'Iter_max' : '{0:.0f}', 't' : '{0:.4f}', 't_max' : '{0:.4f}'}
for col, precision in precisions.items():
    display_df[col] = display_df[col].apply(lambda x: precision.format(x))
display_df.rename(columns={'Algorithm':'Алгоритм', 'std':'СКО', 'sr_0':'Рэф1', 'sr_1':'Рэф2', 'sr_2':'Рэф3', 'sr_3':'Рэф4', 'Ка':'sqrt(Ка)'}, inplace=True)

In [8]:
display_df

,Алгоритм,СКО,Рэф1,Рэф2,Рэф3,Рэф4,sqrt(Ка),Nf,Nf_max,Iter,Iter_max,t,t_max
0,ММП v2,0.0,0.982,0.996,0.996,0.999,0.00,14,32,12,24,0.0024,0.0056
1,Метод N пеленгов,0.0,1.000,1.000,1.000,1.000,0.00,nan,nan,nan,nan,0.0002,0.0013
2,ММП v2,0.1,0.235,0.362,0.366,0.422,0.10,14,32,12,24,0.0023,0.0062
3,Метод N пеленгов,0.1,0.019,0.038,0.040,0.059,48.90,nan,nan,nan,nan,0.0002,0.0006
4,ММП v2,0.2,0.127,0.233,0.238,0.276,0.20,14,32,12,24,0.0024,0.0067
5,Метод N пеленгов,0.2,0.003,0.006,0.006,0.016,79.47,nan,nan,nan,nan,0.0002,0.0004
6,ММП v2,0.3,0.094,0.161,0.168,0.214,0.30,15,32,13,24,0.0024,0.0056
7,Метод N пеленгов,0.3,0.000,0.004,0.004,0.004,89.79,nan,nan,nan,nan,0.0002,0.0010
8,ММП v2,0.5,0.055,0.108,0.112,0.145,0.50,15,32,13,29,0.0026,0.0056
9,Метод N пеленгов,0.5,0.000,0.000,0.000,0.001,98.97,nan,nan,nan,nan,0.0002,0.0004


In [9]:
import docx
doc = docx.Document()
t = doc.add_table(display_df.shape[0]+1, display_df.shape[1])
for j in range(display_df.shape[-1]):
    t.cell(0,j).text = display_df.columns[j]
for i in range(display_df.shape[0]):
    for j in range(display_df.shape[-1]):
        t.cell(i+1,j).text = str(display_df.values[i,j])
doc.save('../tests/test3/{}/Сводная таблица.docx'.format(end_t))